In [ ]:
import numpy as np
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import pickle
import pyotp

In [ ]:
trial_dict = pickle.load(open('./data/trial_info.pkl', 'rb'))
icd_list = pickle.load(open('./data/icd_list.pkl', 'rb'))

In [3]:
trial_dict['NCT00004451']

{'start_date': '1998-03-01',
 'completion_date': nan,
 'condition': ['G20'],
 'summary': " This study will determine whether glucose facilitates memory in healthy elderly people and those with Parkinson's disease. ",
 'criteria': "\n        Inclusion Criteria:\r\n\r\n          -  Three groups of subjects are eligible for this study: Healthy young adults between\r\n             ages 18 and 30; Healthy elderly adults between ages 60 and 90; AND Anyone diagnosed\r\n             with Parkinson's disease\r\n\r\n        Exclusion Criteria:\r\n\r\n          -  Prior surgery to remove part of the stomach\r\n\r\n          -  Diabetes, Addison's, or Cushing's diseases\r\n      "}

In [7]:
trial_condition = {}
for each in trial_dict:
    cur_cond = np.zeros(len(icd_list))
    for each_cond in trial_dict[each]['condition']:
        cur_name = each_cond[:3]
        if cur_name not in icd_list:
            continue
        cur_cond[icd_list.index(cur_name)] = 1
    trial_condition[each] = cur_cond

In [10]:
trial_summary = {}
model = AutoModel.from_pretrained("medicalai/ClinicalBERT", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT", trust_remote_code=True, model_max_length=512, truncation=True, max_length=512)
extractor = pipeline(model=model, tokenizer=tokenizer, task="feature-extraction")
for each in trial_dict:
    cur_sum = trial_dict[each]['summary']
    cur_embd = extractor(cur_sum, truncation=True, max_length=512)
    
    # Generate the sentence embedding
    cur_embd = np.array(cur_embd).squeeze().mean(0)
    trial_summary[each] = cur_embd

c:\Users\Junyi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Junyi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_pr

In [ ]:
pickle.dump(trial_condition, open('./data/trial_condition.pkl', 'wb'))
pickle.dump(trial_summary, open('./data/trial_summary.pkl', 'wb'))